In [11]:
# Author: Asmaa Abdul-Amin
# CRN#: 32016
# This script imports a list of Defense Stocks from a CSV file that can be used to fetch the latest stock prices using the TradingView API.
# The stock prices can then be displayed to the user and saved to a new CSV file with the updated data.

import os
import dotenv
import pandas as pd
import yfinance as yf
import asyncio
import websockets
import json
import matplotlib.pyplot as plt
import mplfinance as mpf


# Load API keys from .env
dotenv.load_dotenv()

ALPACA_API_KEY = os.getenv("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")

# Ensure API keys are set
if not ALPACA_API_KEY or not ALPACA_SECRET_KEY:
    raise ValueError("Missing Alpaca API credentials in .env file!")

# Google Drive CSV Direct Link (Replace with your actual link)
google_drive_csv_url = "https://drive.google.com/uc?id=1_09iFs0UXSXsCOPHFC1H67xQpjdpc1TL"

# Load stock symbols from CSV
df = pd.read_csv(google_drive_csv_url)

# Ensure 'Symbol' column exists
if 'Symbol' not in df.columns:
    raise ValueError("CSV file must contain a 'Symbol' column.")

# Function to fetch historical stock prices from Yahoo Finance
def get_historical_prices(symbol):
    stock = yf.Ticker(symbol)
    hist = stock.history(period="1y")  # Get last 1 year of data
    
    if hist.empty:
        print(f"⚠️ Warning: No data found for {symbol}. Stock might be delisted.")
        return None  # Return None for missing stocks
    
    return hist["Close"]

# Fetch historical prices for all symbols
historical_data = {}

for symbol in df["Symbol"]:
    try:
        data = get_historical_prices(symbol)
        if data is not None:
            historical_data[symbol] = data
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")

# Convert to DataFrame and save as CSV
historical_df = pd.DataFrame(historical_data)

# Ensure we have valid data before proceeding
if historical_df.empty:
    raise ValueError("No valid stock data found. Check if stocks are delisted or API issues.")

# Plot historical prices for top-performing stocks
plt.figure(figsize=(10, 5))

for ticker in historical_data:
    plt.plot(historical_data[ticker], label=ticker)
plt.title("Aerospace/Defense Stock Prices")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.legend()
plt.show()


# Function to calculate stock returns safely
def calculate_returns(historical_prices):
    if len(historical_prices) < 2:  # Ensure we have at least 2 data points
        return None
    return (historical_prices.iloc[-1] - historical_prices.iloc[0]) / historical_prices.iloc[0]

# Identify top-performing stocks based on returns
returns = {
    symbol: calculate_returns(historical_data[symbol])
    for symbol in historical_data if calculate_returns(historical_data[symbol]) > 0.01
}

# Function to calculate stock returns safely
def calculate_returns(historical_prices):
    if len(df) < 2:  # Ensure we have at least 2 data points
        return None
    return (df["Close"].iloc[-1] - df["Close"].iloc[0]) / df["Close"].iloc[0]

# Identify top-performing stocks (returns > 1%)
filtered_stocks = {
    symbol: data for symbol, data in historical_data.items()
    if calculate_returns(data) is not None and calculate_returns(data) > 0.01
}

# Ensure we have valid stocks
if not filtered_stocks:
    print("❌ No stocks with returns greater than 1%. Exiting program.")
    exit()

# Ensure we have valid stocks with returns
if not returns:
    raise ValueError("No stocks with valid returns found.")

# Select top 5 performing stocks
top_stocks = sorted(returns, key=returns.get, reverse=True)[:5]

# Plot historical prices for top-performing stocks
plt.figure(figsize=(10, 5))

for symbol in top_stocks:
    plt.plot(historical_data[symbol], label=symbol)

plt.title("Top Performing Stocks (Based on Returns)")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.legend()
plt.show()


$HEI.A: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")


⚠️ Warning: No data found for HEI.A. Stock might be delisted.


TypeError: '>=' not supported between instances of 'dict' and 'float'

<Figure size 1000x500 with 0 Axes>